This is a Knowledge Graph Question Answering System based on the paper:  
S. Aghaei, E. Raad and A. Fensel, "Question Answering Over Knowledge Graphs: A Case Study in Tourism," in IEEE Access, vol. 10, pp. 69788-69801, 2022, doi: 10.1109/ACCESS.2022.31871

There are some deviations.  Attempts were made to note this, but the knowledge graph (KG) for this project is a couple orders of magnitude smaller (with about 400 facts/triples).  Due to time contraints shortcuts may have been taken.

This is just an interface to show the code working.  The code for this is in Python under the kgqas/ folder.  Every field can be ran.

# Zoning Question Answering

This is just a simple example showing this work.

In [1]:
%%time
# first time this is run Stanza will download models, this will take about 1 minute.
import semantic_parsing
sem_par = semantic_parsing.SemanticParsingClass()

2022-12-03 00:10:21 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-12-03 00:10:22 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-12-03 00:10:22 INFO: Use device: cpu
2022-12-03 00:10:22 INFO: Loading: tokenize
2022-12-03 00:10:22 INFO: Loading: pos
2022-12-03 00:10:23 INFO: Loading: lemma
2022-12-03 00:10:23 INFO: Loading: depparse
2022-12-03 00:10:23 INFO: Done loading processors!
2022-12-03 00:10:23.437 | INFO     | question_classification:load_model:296 - Loading XGBoost model: question_classification_model.ubj


CPU times: user 4.8 s, sys: 1.67 s, total: 6.47 s
Wall time: 42.2 s


Below are a few random questions you could ask, rerun the cell to generate new ones or change the `n` value to get more/fewer questions.  

These are from the templates in `generate_template.py` that execute SPARQL to get the answers.  The question corpus has 900 questions in all (162 questions were excluded because of bad answers those would take time to fix them, so the total number of questions is 1062 before exclusion).

In [2]:
%%time
itr = semantic_parsing.get_random_questions_answers(sem_par.generate_filtered_corpus(), n=5)
for qa in itr:
    print(f"Question: {qa['question']}")
    if qa['answer'] is True:
        print(f"Answer: Yes")
    else:
        print(f"Answer: {qa['answer']}")

Question: Are family and group day care facilities permitted in a C2 zoning district?
Answer: Yes
Question: Are public libraries permitted in a R3 zoning district?
Answer: Yes
Question: I would like to build restaurants.  Which zoning districts permits this use?
Answer: ['C1', 'C2', 'C3', 'C4']
Question: Are temporary buildings permitted in a R1 zoning district?
Answer: Yes
Question: I would like to build agricultural Use.  Which zoning districts permits this use?
Answer: ['A2']
CPU times: user 1.38 s, sys: 9.62 ms, total: 1.39 s
Wall time: 1.39 s


## Zoning KGQAS
Below is the Zoning KGQAS system working.

Copy the above questions.  Try to use some variation on a question.  Here's the [International Zoning Code 2021](https://codes.iccsafe.org/content/IZC2021P1/arrangement-and-format-of-the-2021-izc), the source document, if you wish to try to look up a question.  Only questions about dimensional requirements and permitted uses can be answerred. 

Example questions:
* A question could be "Are quarries allowed?" is in the dataset, but mining is not so "Is mining allowed?" would be a similar use (the answer should be "Yes").  (It correctly identifies that quarries and mining are similar words.)

* The International Zoning Code uses "financial services", so asking "Is a bank allowed?" a similar use.  (the answer should be "Yes").

I asked a question, which off the top of my head I think have a No answer: 
* "Is a trailer park allowed?"  It answers "Yes," it is matching "publicly owned and operated parks".  
* Then, I ask "Which zoning districts allow trailer parks?", again it answers "Yes," matching the same Question Classification.  It matched the same question An answer that would I could beleive would be "R1" (or some other R District).  I guess I'll have to accept that Zoing KGQAS is a people pleaser.   Kidding aside, it has overfitted because only questions with Yes answers were sent to it in training.
* I ask "Is a marijuana dispensery allowed?", it matches that use as "A2" which is an Agricultural zoning district.  It gives the correct answer of "No", but it doesn't know that it should be "No".
* I ask "I would like to build a marijuana dispensery.  Which zoning districts permits this use?"  It correctly gives an empty list [].

This is thought provoking, but I'd like more experience with this before I serious suggest ways to fix this.  Off the top of my head, perhaps this is SBERT or the choice of Levenstein Edit Distance.  Also, perhaps you occationally need this to match a No/null case, when the question is too far from a Yes option.

In [3]:
%%time
# question = "What is the minimum lot depth in the C3 zoning district?"   # Answer: 100 [ft_i]
# question = "Are quarries allowed?"  # A: Yes
question = "Is mining allowed?"       # A: Yes
# question = "Is a bank allowed?"     # A: Yes
# question = "Is a trailer park allowed?"   # A: Yes   (Seems incorrect)
# question = "Which zoning districts allow trailer parks?"  # A: Yes   (Seems incorrect, incorrect type of answer)
# question = "Is a marijuana dispensery allowed?"  # A: No
# question = "I would like to build a marijuana dispensery.  Which zoning districts permits this use?"  # A: []

answer = sem_par.classify(question)

print(f"Question: {question}")
if answer is True:
    print(f"Answer: Yes")
elif answer is False:
    print(f"Answer: No")
else:
    print(f"Answer: {answer}")

2022-12-02 18:26:47.681 | INFO     | question_classification:load_model:292 - Loading XGBoost model: question_classification_model.ubj


[[ 83570  85230  85717 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[2]
template name: template_use_1var_yn_answer
SPARQL TEMPLATE: 
ASK {
        ?zoning :permitsUse "${use}" .
}

VARIABLES: ('use',)
RELATIONS: None
SLOTS: {}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(1.7109150355375984, 'quarries')]
SLOTS: {'use': 'quarries'}
SPARQL:  
ASK {
        ?zoning :permitsUse "quarries" .
}

====== The answer is ======
Yes
Question: Is mining allowed?
Answer: Yes
CPU times: user 1.29 s, sys: 87.9 ms, total: 1.37 s
Wall time: 1.82 s


# Metrics

The below cell has problems running without error.

In [2]:
%%time
# I had problems getting this cell to run without error.
# These take 44 minutes to run on CPU for 900 questions (1 question every 3 seconds) 
semantic_parsing.measure_accuracy()


2022-12-03 00:12:29.744 | INFO     | semantic_parsing:measure_accuracy:139 - measuring the accuracy of Zoning KGQAS
2022-12-03 00:12:29 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-12-03 00:12:30 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-12-03 00:12:30 INFO: Use device: cpu
2022-12-03 00:12:30 INFO: Loading: tokenize
2022-12-03 00:12:30 INFO: Loading: pos
2022-12-03 00:12:30 INFO: Loading: lemma
2022-12-03 00:12:30 INFO: Loading: depparse
2022-12-03 00:12:30 INFO: Done loading processors!
2022-12-03 00:12:30.955 | INFO     | question_classification:load_model:296 - Loading XGBoost model: question_classification_model.ubj


question_corpus_filt: 900
[[ 95162  34359  33523  33906  96234  33690  34359  33908  33906  35124
   35821 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[0]


2022-12-03 00:12:36.676 | INFO     | relation_extraction:extract:191 - Loading MLPClassifier Model: relation_extraction_model.pickle


template name: template_dimreg_2var_m_answer
SPARQL TEMPLATE: 
SELECT ?regulation_value

WHERE {
        ?zoning $regulation_predicate ?regulation_value ;
                rdfs:label "${zoning}" .
}

VARIABLES: ('regulation_predicate', 'regulation_text', 'zoning')
RELATIONS: ['minimum lot size']
SLOTS: {'regulation_predicate': ':minLotSize'}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(2.0510093258166675, 'R1c')]
SLOTS: {'regulation_predicate': ':minLotSize', 'zoning': 'R1c'}
SPARQL:  
SELECT ?regulation_value

WHERE {
        ?zoning :minLotSize ?regulation_value ;
                rdfs:label "R1c" .
}

====== The answer is ======
Results OBJ vars: regulation_value
10000 [sft_i]
[[ 95162  34359  33523  96234  33690  34359  33908  33906  35124  96805
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[0]
te

...

template name: template_dimreg_2var_m_answer
SPARQL TEMPLATE: 
SELECT ?regulation_value

WHERE {
        ?zoning $regulation_predicate ?regulation_value ;
                rdfs:label "${zoning}" .
}

VARIABLES: ('regulation_predicate', 'regulation_text', 'zoning')
RELATIONS: ['maximum building height']
SLOTS: {'regulation_predicate': ':maxBuildingHeight'}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(1.8920791458703805, 'R1b')]
SLOTS: {'regulation_predicate': ':maxBuildingHeight', 'zoning': 'R1b'}
SPARQL:  
SELECT ?regulation_value

WHERE {
        ?zoning :maxBuildingHeight ?regulation_value ;
                rdfs:label "R1b" .
}

====== The answer is ======
Results OBJ vars: regulation_value
35 [ft_i]
[[ 95162  34359  33523  33906  96234  33690  34359  33908  33906  35124
   96805 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 12000

template name: template_dimreg_2var_m_answer
SPARQL TEMPLATE: 
SELECT ?regulation_value

WHERE {
        ?zoning $regulation_predicate ?regulation_value ;
                rdfs:label "${zoning}" .
}

VARIABLES: ('regulation_predicate', 'regulation_text', 'zoning')
RELATIONS: ['minimum lot width']
SLOTS: {'regulation_predicate': ':minLotWidth'}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(2.0681350474149793, 'R2b')]
SLOTS: {'regulation_predicate': ':minLotWidth', 'zoning': 'R2b'}
SPARQL:  
SELECT ?regulation_value

WHERE {
        ?zoning :minLotWidth ?regulation_value ;
                rdfs:label "R2b" .
}

====== The answer is ======
Results OBJ vars: regulation_value
60 [ft_i]
[[ 95162  34359  33523  33906  96234  33690  34359  33908  33906  35124
   96805 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[0]
te

template name: template_dimreg_2var_m_answer
SPARQL TEMPLATE: 
SELECT ?regulation_value

WHERE {
        ?zoning $regulation_predicate ?regulation_value ;
                rdfs:label "${zoning}" .
}

VARIABLES: ('regulation_predicate', 'regulation_text', 'zoning')
RELATIONS: ['minimum lot depth']
SLOTS: {'regulation_predicate': ':minLotDepth'}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(2.1283948143804166, 'R2a')]
SLOTS: {'regulation_predicate': ':minLotDepth', 'zoning': 'R2a'}
SPARQL:  
SELECT ?regulation_value

WHERE {
        ?zoning :minLotDepth ?regulation_value ;
                rdfs:label "R2a" .
}

====== The answer is ======
Results OBJ vars: regulation_value
70 [ft_i]
[[ 95162  34359  33523  33523  96234  33690  34359  33908  33906  35124
   96805 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[0]
te

template name: template_dimreg_2var_m_answer
SPARQL TEMPLATE: 
SELECT ?regulation_value

WHERE {
        ?zoning $regulation_predicate ?regulation_value ;
                rdfs:label "${zoning}" .
}

VARIABLES: ('regulation_predicate', 'regulation_text', 'zoning')
RELATIONS: ['minimum front setback']
SLOTS: {'regulation_predicate': ':minFrontSetback'}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(2.045894323521015, 'R3a')]
SLOTS: {'regulation_predicate': ':minFrontSetback', 'zoning': 'R3a'}
SPARQL:  
SELECT ?regulation_value

WHERE {
        ?zoning :minFrontSetback ?regulation_value ;
                rdfs:label "R3a" .
}

====== The answer is ======
Results OBJ vars: regulation_value
15 [ft_i]
[[ 95162  34359  33523  33906  96234  33690  34359  33908  33906  35124
   96805 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 

KeyboardInterrupt: 

Instead, I can get a subset of the date to run and get F1 and Accuracy.  It gives an accuracy of 100.0 and an f1 score of 100.0 (yes, the same score that some dictators "win" elections by).  I do not think these metrics are reliable.  There are concerns of overfitting.  (The questions that are pulls seem to all from the same template, which seems problematic for producing randomized output. Perhaps the chained generators aren't allowing for randomized output to occur.)

This entire project a very simple example.  The questions are quite simple with regard matching up with the templates.  There is very little of the input that it hasn't seen before in this training.

In [4]:
%%time
# this subset runs in about 5 minutes
semantic_parsing.measure_accuracy(subset=90, randomized_subset=True)

2022-12-03 00:18:47.202 | INFO     | semantic_parsing:measure_accuracy:139 - measuring the accuracy of Zoning KGQAS
2022-12-03 00:18:47 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-12-03 00:18:47 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-12-03 00:18:47 INFO: Use device: cpu
2022-12-03 00:18:47 INFO: Loading: tokenize
2022-12-03 00:18:47 INFO: Loading: pos
2022-12-03 00:18:48 INFO: Loading: lemma
2022-12-03 00:18:48 INFO: Loading: depparse
2022-12-03 00:18:48 INFO: Done loading processors!
2022-12-03 00:18:48.405 | INFO     | question_classification:load_model:296 - Loading XGBoost model: question_classification_model.ubj
2022-12-03 00:18:50.312 | INFO     | semantic_parsing:measure_accuracy:149 - measuring randomized subset of size: 90


question_corpus_filt: 900
[[ 95162  34359  33523  33906  96234  33690  34359  33908  33906  35124
   96805 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[0]


2022-12-03 00:18:53.321 | INFO     | relation_extraction:extract:191 - Loading MLPClassifier Model: relation_extraction_model.pickle


template name: template_dimreg_2var_m_answer
SPARQL TEMPLATE: 
SELECT ?regulation_value

WHERE {
        ?zoning $regulation_predicate ?regulation_value ;
                rdfs:label "${zoning}" .
}

VARIABLES: ('regulation_predicate', 'regulation_text', 'zoning')
RELATIONS: ['minimum lot depth']
SLOTS: {'regulation_predicate': ':minLotDepth'}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(2.0593242093517508, 'R3b')]
SLOTS: {'regulation_predicate': ':minLotDepth', 'zoning': 'R3b'}
SPARQL:  
SELECT ?regulation_value

WHERE {
        ?zoning :minLotDepth ?regulation_value ;
                rdfs:label "R3b" .
}

====== The answer is ======
Results OBJ vars: regulation_value
70 [ft_i]
[[ 95162  34359  33523  33906  96234  33690  34359  33908  33906  35124
   96805 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[0]
te

...

[[ 95162  34359  33523  33906  96234  33690  34359  33908  33906  35124
   96805 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[0]
template name: template_dimreg_2var_m_answer
SPARQL TEMPLATE: 
SELECT ?regulation_value

WHERE {
        ?zoning $regulation_predicate ?regulation_value ;
                rdfs:label "${zoning}" .
}

VARIABLES: ('regulation_predicate', 'regulation_text', 'zoning')
RELATIONS: ['minimum lot depth']
SLOTS: {'regulation_predicate': ':minLotDepth'}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(2.105695886629, 'R2b')]
SLOTS: {'regulation_predicate': ':minLotDepth', 'zoning': 'R2b'}
SPARQL:  
SELECT ?regulation_value

WHERE {
        ?zoning :minLotDepth ?regulation_value ;
                rdfs:label "R2b" .
}

====== The answer is ======
Results OBJ vars: regulation_value
70 [ft_i]
[[ 951

template name: template_dimreg_2var_m_answer
SPARQL TEMPLATE: 
SELECT ?regulation_value

WHERE {
        ?zoning $regulation_predicate ?regulation_value ;
                rdfs:label "${zoning}" .
}

VARIABLES: ('regulation_predicate', 'regulation_text', 'zoning')
RELATIONS: ['minimum side setback']
SLOTS: {'regulation_predicate': ':minSideSetback'}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(2.08020846991883, 'R2a')]
SLOTS: {'regulation_predicate': ':minSideSetback', 'zoning': 'R2a'}
SPARQL:  
SELECT ?regulation_value

WHERE {
        ?zoning :minSideSetback ?regulation_value ;
                rdfs:label "R2a" .
}

====== The answer is ======
Results OBJ vars: regulation_value
5 [ft_i]
[[ 95162  34359  33523  33523  96234  33690  34359  33908  33906  35124
   96805 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000

template name: template_dimreg_2var_m_answer
SPARQL TEMPLATE: 
SELECT ?regulation_value

WHERE {
        ?zoning $regulation_predicate ?regulation_value ;
                rdfs:label "${zoning}" .
}

VARIABLES: ('regulation_predicate', 'regulation_text', 'zoning')
RELATIONS: ['minimum lot size']
SLOTS: {'regulation_predicate': ':minLotSize'}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(2.0837302902380586, 'R1d')]
SLOTS: {'regulation_predicate': ':minLotSize', 'zoning': 'R1d'}
SPARQL:  
SELECT ?regulation_value

WHERE {
        ?zoning :minLotSize ?regulation_value ;
                rdfs:label "R1d" .
}

====== The answer is ======
Results OBJ vars: regulation_value
6000 [sft_i]
[[ 95162  34359  33523  33523  96234  33690  34359  33908  33906  35124
   96805 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[0]
tem

template name: template_dimreg_2var_m_answer
SPARQL TEMPLATE: 
SELECT ?regulation_value

WHERE {
        ?zoning $regulation_predicate ?regulation_value ;
                rdfs:label "${zoning}" .
}

VARIABLES: ('regulation_predicate', 'regulation_text', 'zoning')
RELATIONS: ['minimum front setback']
SLOTS: {'regulation_predicate': ':minFrontSetback'}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(1.9945465361953434, 'R3b')]
SLOTS: {'regulation_predicate': ':minFrontSetback', 'zoning': 'R3b'}
SPARQL:  
SELECT ?regulation_value

WHERE {
        ?zoning :minFrontSetback ?regulation_value ;
                rdfs:label "R3b" .
}

====== The answer is ======
Results OBJ vars: regulation_value
15 [ft_i]
[[ 95162  34359  33523  33906  96234  33690  34359  33908  33906  35124
   96805 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000

template name: template_dimreg_2var_m_answer
SPARQL TEMPLATE: 
SELECT ?regulation_value

WHERE {
        ?zoning $regulation_predicate ?regulation_value ;
                rdfs:label "${zoning}" .
}

VARIABLES: ('regulation_predicate', 'regulation_text', 'zoning')
RELATIONS: ['minimum lot depth']
SLOTS: {'regulation_predicate': ':minLotDepth'}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(2.0013936548194162, 'R1a')]
SLOTS: {'regulation_predicate': ':minLotDepth', 'zoning': 'R1a'}
SPARQL:  
SELECT ?regulation_value

WHERE {
        ?zoning :minLotDepth ?regulation_value ;
                rdfs:label "R1a" .
}

====== The answer is ======
Results OBJ vars: regulation_value
150 [ft_i]
[[ 95162  34359  33523  96234  33690  34359  33908  33906  35124  96805
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[0]
t

Here's a slightly bigger example of 270 questions.  This gives an error.  There is a problem with matching the answers with the gold_answers. 

In [5]:
%%time
# anticipate 15 minute runtime
# it gives ERRORS
semantic_parsing.measure_accuracy(subset=270, randomized_subset=True)

2022-12-03 00:46:54.504 | INFO     | semantic_parsing:measure_accuracy:139 - measuring the accuracy of Zoning KGQAS
2022-12-03 00:46:54 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-12-03 00:46:55 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-12-03 00:46:55 INFO: Use device: cpu
2022-12-03 00:46:55 INFO: Loading: tokenize
2022-12-03 00:46:55 INFO: Loading: pos
2022-12-03 00:46:55 INFO: Loading: lemma
2022-12-03 00:46:55 INFO: Loading: depparse
2022-12-03 00:46:55 INFO: Done loading processors!
2022-12-03 00:46:55.759 | INFO     | question_classification:load_model:296 - Loading XGBoost model: question_classification_model.ubj
2022-12-03 00:46:57.781 | INFO     | semantic_parsing:measure_accuracy:149 - measuring randomized subset of size: 270


question_corpus_filt: 900
[[ 76836  75191  76675  77559  36295  33906  33906  36678  77041  77401
   37161  36678  90691  34359  90900  91261 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[1]
template name: template_use_1var_m_answer
SPARQL TEMPLATE: 
SELECT ?zoning_label

WHERE {
        ?zoning :permitsUse "${use}" .
        ?zoning rdfs:label ?zoning_label .
}

VARIABLES: ('use',)
RELATIONS: None
SLOTS: {}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(3.1622326959403684, 'automotive motor fuel dispensing facilities')]
SLOTS: {'use': 'automotive motor fuel dispensing facilities'}
SPARQL:  
SELECT ?zoning_label

WHERE {
        ?zoning :permitsUse "automotive motor fuel dispensing facilities" .
        ?zoning rdfs:label ?zoning_label .
}

====== The answer is ======
Results OBJ vars: zoning_label
C1
C2
C3
C4
[[ 95162  34359  33523  33523  962

2022-12-03 00:47:05.957 | INFO     | relation_extraction:extract:191 - Loading MLPClassifier Model: relation_extraction_model.pickle


template name: template_dimreg_2var_m_answer
SPARQL TEMPLATE: 
SELECT ?regulation_value

WHERE {
        ?zoning $regulation_predicate ?regulation_value ;
                rdfs:label "${zoning}" .
}

VARIABLES: ('regulation_predicate', 'regulation_text', 'zoning')
RELATIONS: ['minimum front setback']
SLOTS: {'regulation_predicate': ':minFrontSetback'}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(2.045894323521015, 'R3a')]
SLOTS: {'regulation_predicate': ':minFrontSetback', 'zoning': 'R3a'}
SPARQL:  
SELECT ?regulation_value

WHERE {
        ?zoning :minFrontSetback ?regulation_value ;
                rdfs:label "R3a" .
}

====== The answer is ======
Results OBJ vars: regulation_value
15 [ft_i]
[[ 76836  75191  76675  77559  36295  77041  77401  37161  36678  90691
   34359  90900  91261 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 

...

template name: template_use_1var_m_answer
SPARQL TEMPLATE: 
SELECT ?zoning_label

WHERE {
        ?zoning :permitsUse "${use}" .
        ?zoning rdfs:label ?zoning_label .
}

VARIABLES: ('use',)
RELATIONS: None
SLOTS: {}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(3.336252214645338, 'rehabilitation centers')]
SLOTS: {'use': 'rehabilitation centers'}
SPARQL:  
SELECT ?zoning_label

WHERE {
        ?zoning :permitsUse "rehabilitation centers" .
        ?zoning rdfs:label ?zoning_label .
}

====== The answer is ======
Results OBJ vars: zoning_label
C2
C3
C4
[[ 76836  75191  76675  77559  77041  77401  37161  36678  90691  34359
   90900  91261 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[1]
template name: template_use_1var_m_answer
SPARQL TEMPLATE: 
SELECT ?zoning_label

WHERE {
        ?zoning :permitsUse "${use}" 

template name: template_use_1var_m_answer
SPARQL TEMPLATE: 
SELECT ?zoning_label

WHERE {
        ?zoning :permitsUse "${use}" .
        ?zoning rdfs:label ?zoning_label .
}

VARIABLES: ('use',)
RELATIONS: None
SLOTS: {}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(3.011520849566416, 'libraries')]
SLOTS: {'use': 'libraries'}
SPARQL:  
SELECT ?zoning_label

WHERE {
        ?zoning :permitsUse "libraries" .
        ?zoning rdfs:label ?zoning_label .
}

====== The answer is ======
Results OBJ vars: zoning_label
C1
C2
C3
C4
[[ 95162  34359  33523  33523  96234  33690  34359  33908  33906  35124
   96805 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[0]
template name: template_dimreg_2var_m_answer
SPARQL TEMPLATE: 
SELECT ?regulation_value

WHERE {
        ?zoning $regulation_predicate ?regulation_value ;
        

template name: template_dimreg_2var_m_answer
SPARQL TEMPLATE: 
SELECT ?regulation_value

WHERE {
        ?zoning $regulation_predicate ?regulation_value ;
                rdfs:label "${zoning}" .
}

VARIABLES: ('regulation_predicate', 'regulation_text', 'zoning')
RELATIONS: ['minimum lot depth']
SLOTS: {'regulation_predicate': ':minLotDepth'}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(2.0593242093517508, 'R3b')]
SLOTS: {'regulation_predicate': ':minLotDepth', 'zoning': 'R3b'}
SPARQL:  
SELECT ?regulation_value

WHERE {
        ?zoning :minLotDepth ?regulation_value ;
                rdfs:label "R3b" .
}

====== The answer is ======
Results OBJ vars: regulation_value
70 [ft_i]
[[ 76836  75191  76675  77559  77040  36499  36678  36295  34117  82948
   38795  36678  82585  82949  77401  37161  36678  90691  34359  90900
   91261 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[1]
te

template name: template_use_1var_m_answer
SPARQL TEMPLATE: 
SELECT ?zoning_label

WHERE {
        ?zoning :permitsUse "${use}" .
        ?zoning rdfs:label ?zoning_label .
}

VARIABLES: ('use',)
RELATIONS: None
SLOTS: {}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(2.2586273927208915, 'convenience and neighborhood commercial centers (excluding wholesale sales)')]
SLOTS: {'use': 'convenience and neighborhood commercial centers (excluding wholesale sales)'}
SPARQL:  
SELECT ?zoning_label

WHERE {
        ?zoning :permitsUse "convenience and neighborhood commercial centers (excluding wholesale sales)" .
        ?zoning rdfs:label ?zoning_label .
}

====== The answer is ======
Results OBJ vars: zoning_label
C1
C2
C3
C4
[[ 76836  75191  76675  77559  33523  33906  77040  35822  34116  33727
   34116  77401  37161  36678  90691  34359  90900  91261 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 12000

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

# Training

Training is pretty fast.  Probably, not a great job of training.  No cross validation done on the data.  Trained model binaries are already in the folder, so this step isn't necessary (also why this is at the bottom).

In [13]:
%%time
# takes apx. 2-3 minutes CPU time
# on first time run Stanza has to download some packages
import semantic_parsing
sem_par = semantic_parsing.SemanticParsingClass()
sem_par.train_all()

2022-12-03 04:20:32 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-12-03 04:20:33 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-12-03 04:20:33 INFO: Use device: cpu
2022-12-03 04:20:33 INFO: Loading: tokenize
2022-12-03 04:20:33 INFO: Loading: pos
2022-12-03 04:20:33 INFO: Loading: lemma
2022-12-03 04:20:33 INFO: Loading: depparse
2022-12-03 04:20:33 INFO: Done loading processors!
2022-12-03 04:20:33.698 | INFO     | question_classification:load_model:296 - Loading XGBoost model: question_classification_model.ubj
2022-12-03 04:20:36.338 | INFO     | question_classification:train2:215 - 1. Question Classifier training begun. ================
2022-12-03 04:23:30.209 | INFO    

Number of Questions Encoded: 900
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

/home/micah/anaconda3/envs/NLP-SPARQL/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['A1', 'A2', 'C1', 'C2', 'C3', 'C4', 'FI1', 'FI2', 'FI3', 'R1', 'R1a', 'R1b', 'R1c', 'R1d', 'R2', 'R2a', 'R2b', 'R3', 'R3a', 'R3b', 'agricultural Use', 'alcohol manufacturing', 'amusement centers (including bowling alleys, golf driving ranges miniature golf courses, ice rinks, pool and billiard halls, and similar recreational uses)', 'auto body shops', 'auto-dismantling yards', 'automotive motor fuel dispensing facilities', 'automotive sales', 'automotive self-service motor fuel dispensing facilities', 'boat storage yards', 'breweries', 'building material supply sales (wholesale and retail)', 'buildings accessory to the permitted uses (including private garages and accessory living quarters)', 'business services', 'carpet manufacturing', 'commercial bakeries', 'commercial printing and publishing', 'community commercial centers (including wholesale and re

Iteration 1, loss = 6.17999764
Iteration 2, loss = 5.75437268
Iteration 3, loss = 5.35139345
Iteration 4, loss = 4.92773495
Iteration 5, loss = 4.47628013
Iteration 6, loss = 3.99959766
Iteration 7, loss = 3.51782247
Iteration 8, loss = 3.05066730
Iteration 9, loss = 2.61346243
Iteration 10, loss = 2.22000400
Iteration 11, loss = 1.88176420
Iteration 12, loss = 1.60006071
Iteration 13, loss = 1.37263206
Iteration 14, loss = 1.19277745
Iteration 15, loss = 1.05243261
Iteration 16, loss = 0.94549576
Iteration 17, loss = 0.86254460
Iteration 18, loss = 0.79877202
Iteration 19, loss = 0.74952663
Iteration 20, loss = 0.71037358
Iteration 21, loss = 0.67782238
Iteration 22, loss = 0.65127408
Iteration 23, loss = 0.62882002
Iteration 24, loss = 0.60982883
Iteration 25, loss = 0.59262604
Iteration 26, loss = 0.57753201
Iteration 27, loss = 0.56370718
Iteration 28, loss = 0.55090301
Iteration 29, loss = 0.53877991
Iteration 30, loss = 0.52767392
Iteration 31, loss = 0.51694736
Iteration 32, los

Iteration 258, loss = 0.04899604
Iteration 259, loss = 0.04853297
Iteration 260, loss = 0.04804959
Iteration 261, loss = 0.04757908
Iteration 262, loss = 0.04717169
Iteration 263, loss = 0.04669554
Iteration 264, loss = 0.04627204
Iteration 265, loss = 0.04580346
Iteration 266, loss = 0.04539615
Iteration 267, loss = 0.04499241
Iteration 268, loss = 0.04448975
Iteration 269, loss = 0.04404409
Iteration 270, loss = 0.04363118
Iteration 271, loss = 0.04321724
Iteration 272, loss = 0.04278140
Iteration 273, loss = 0.04235718
Iteration 274, loss = 0.04196729
Iteration 275, loss = 0.04155101
Iteration 276, loss = 0.04116054
Iteration 277, loss = 0.04076983
Iteration 278, loss = 0.04037684
Iteration 279, loss = 0.04001214
Iteration 280, loss = 0.03963749
Iteration 281, loss = 0.03928123
Iteration 282, loss = 0.03888206
Iteration 283, loss = 0.03855571
Iteration 284, loss = 0.03816913
Iteration 285, loss = 0.03782132
Iteration 286, loss = 0.03746150
Iteration 287, loss = 0.03713385
Iteration 

# Semantic Parsing 

Here's the output of each of these phases during classification.  This code runs, but it is really just intended to illustrate some of what is going on in the classes being imported.

In [5]:
# These questions will be used to go through the steps of this process, as applicable.
questions = ['Are auto-dismantling yards permitted?',
             'Is the maximum building height for a property in the C3 zoning district 50 feet?',
             'Which zoning districts allow physical fitness centers?',
             'Are salt works allowed in a FI3 zoning district?',
             'What is the maximum building height in the R2 zoning district?',]

## 1) Question Classification
This phase identifies the template number that the question matches based on a vectors created from \<POS head tagging, dependecy, POS dependent word\>  The matrix shows the embeding for a question.  XGBoost is used for this.

One of the results below is incorrect (it should be template_dimreq_*):
```
Question: Is the maximum building height for a property in the C3 zoning district 50 feet? 	 template_number = 1 
 	 template_name = template_use_1var_m_answer
```

In [15]:
from question_classification import QuestionClassification
qc = QuestionClassification()
model = qc.load_model()
# This is the number converted into the template's name
classified_template_names = []
for q in questions:
    # template number from XGBoost
    template_number = qc.classify(q, model)
    # convert the template's number into a name
    template_name = qc.classification_number_to_template_name(template_number)
    classified_template_names.append(template_name)
    print(f"Question: {q} \t template_number = {template_number} \n \t template_name = {template_name}")
    print('-------------------------------------------------------------------------')

2022-12-03 04:25:47 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-12-03 04:25:48 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-12-03 04:25:48 INFO: Use device: cpu
2022-12-03 04:25:48 INFO: Loading: tokenize
2022-12-03 04:25:48 INFO: Loading: pos
2022-12-03 04:25:48 INFO: Loading: lemma
2022-12-03 04:25:48 INFO: Loading: depparse
2022-12-03 04:25:49 INFO: Done loading processors!
2022-12-03 04:25:49.144 | INFO     | question_classification:load_model:296 - Loading XGBoost model: question_classification_model.ubj
2022-12-03 04:25:49.150 | INFO     | question_classification:load_model:296 - Loading XGBoost model: question_classification_model.ubj


[[ 83569  33906  35823  36678  85231  85717 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[2]
Question: Are auto-dismantling yards permitted? 	 template_number = 2 
 	 template_name = template_use_1var_yn_answer
-------------------------------------------------------------------------
[[ 36950  34359  33523  33906  38022  33690  34359  35124  33690  34359
   33908  33906  36678  38138  38593 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000
  120000 120000 120000 120000 120000 120000 120000 120000 120000 120000]]
[1]
Question: Is the maximum building height for a property in the C3 zoning district 50 feet? 	 template_number = 1 
 	 template_name = template_use_1var_m_answer
------------------------------------------------------------

## 2) Entity Linking and Class Linking
I'm just showing one question because there is too much output from this question.  This is a list of string similarity scores with the token that it matches.

The top two entities in the list `FI3` and `salt works` are the entities that need to be identified in the question.


In [25]:
import pprint
from entity_class_linking import EntityClassLinking

similarity_scores = []
ecl = EntityClassLinking(verbose=False)
for q in questions: 
    ngram = ecl.ngram_collection(q)
    similarity_score = ecl.string_similarity_score(ngram)
    similarity_scores.append(similarity_score)

print(f"Question: {questions[3]}")
pprint.pprint(f"Similarity Score: {similarity_scores[3]}")

Question: Are salt works allowed in a FI3 zoning district?
("Similarty Score: [(1.5728168031884948, 'FI3'), (1.5568653406766861, 'salt "
 "works'), (0.5940250986658693, 'FI2'), (0.5077651345301475, 'FI1'), "
 "(0.1788948217158074, 'acre'), (0.17517721735216904, 'A3'), "
 "(0.1519791417009425, 'A2'), (0.13190111441469937, 'C3'), "
 "(0.12461983485574252, 'dwellings'), (0.11390362703271624, 'storage yards'), "
 "(0.10602401229650368, 'R3'), (0.10005524169571235, 'R3a'), "
 "(0.09888981242046722, 'A1'), (0.08239931702437335, 'R3b'), "
 "(0.08195433402703464, 'police and fire department stations'), "
 "(0.08149525386310359, 'public parking lots'), (0.07998294243036931, "
 "'regional recycling center'), (0.0790904900476317, 'warehouses'), "
 "(0.07107681901040906, '2'), (0.07017845243001852, 'recreation centers'), "
 "(0.06766887391369414, '1'), (0.06476261805874774, 'acres per dwelling "
 "unit'), (0.06142906634392661, '4'), (0.05884447602967138, 'regional "
 "commercial centers (including

## 3) Relation Extraction

This step identified predicates, which are relationships.  It correctly identifies that the `maximum building height`.

In [41]:
import kg_helper
from relation_extraction import RelationExtraction

relation_list = []
relex = RelationExtraction()
for i, q in enumerate(questions):
    template_dict = kg_helper.get_template(classified_template_names[i])
    if template_dict['knowledge_graph'] == 'permitted_uses':
        # skip, permitted_use templates have an assumed relationship that doesn't need to be extracted.
        relation_list.append(None)
        continue
    else:
        k = 1  # the number of relationships to find, this KG only needs 1 to answer the questions
        print(f"Question {i}: {q}, k: {k}")
        result = relex.extract(q, k)
        relation_list.append(result)
        print(result)

2022-12-03 04:52:57.953 | INFO     | relation_extraction:extract:191 - Loading MLPClassifier Model: relation_extraction_model.pickle


Question 4: What is the maximum building height in the R2 zoning district?, k: 1
['maximum building height']


## 4) Slot Filling and Query Execution
This cell relies upon cells 1-3 being ran before it to be able to answer the question.  About half of the questions are answered correctly.  You get to see the SPARQL template and the SPARQL query, too in this.

In [48]:
from slot_filling_query_execution import SlotFillingQueryExecution
# print(len(relation_list))
sfqe = SlotFillingQueryExecution()
for i, c_template in enumerate(classified_template_names):
    res = sfqe.slot_fill_query_execute(c_template, similarity_scores[i], relation_list[i])
    print(f"Question: {questions[i]}")
    if res is True:
        print('Answer: Yes')
    elif res is False:
        print('Answer: No')
    else:
        print(f'Answer: {res}')

template name: template_use_1var_yn_answer
SPARQL TEMPLATE: 
ASK {
        ?zoning :permitsUse "${use}" .
}

VARIABLES: ('use',)
RELATIONS: None
SLOTS: {}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(2.969672117986351, 'auto-dismantling yards')]
SLOTS: {'use': 'auto-dismantling yards'}
SPARQL:  
ASK {
        ?zoning :permitsUse "auto-dismantling yards" .
}

====== The answer is ======
Yes
Question: Are auto-dismantling yards permitted?
Answer: Yes
template name: template_use_1var_m_answer
SPARQL TEMPLATE: 
SELECT ?zoning_label

WHERE {
        ?zoning :permitsUse "${use}" .
        ?zoning rdfs:label ?zoning_label .
}

VARIABLES: ('use',)
RELATIONS: None
SLOTS: {}
num_entity_slots: 1
SIMILARITY SCORES for num_entity_slots: [(1.1226739442659794, '50')]
SLOTS: {'use': '50'}
SPARQL:  
SELECT ?zoning_label

WHERE {
        ?zoning :permitsUse "50" .
        ?zoning rdfs:label ?zoning_label .
}

====== The answer is ======
Results OBJ vars: zoning_label
Question: Is the ma

In [ ]:
That's it.  That's the process.  Please check out the Python code or the orinif you want to learn